# Seq2Seq

あるシーケンスから別のシーケンスへの変換を行うSeq2Seqというモデルを学び、機械翻訳へ応用する。

In [1]:
import os; os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings; warnings.filterwarnings("ignore")
import math
import random
import copy

import tensorflow as tf
import tensorflow_datasets as tfds
import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import (
    pad_sequence,
    pack_padded_sequence,
    pad_packed_sequence,
)
from dlprog import train_progress

In [2]:
prog = train_progress(
    width=20,
    with_test=True,
    label="ppl train",
    round=2,
    agg_fn=lambda s, w: math.exp(s / w)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')


---

## 条件付き言語モデル

言語モデルに文脈以外の条件を付与する。

### 言語モデルへの条件付け

これまでの言語モデルは文脈を条件とした確率モデルであった。

$$
p(w_t|w_{<t})
$$

ここで、文脈以外の条件を追加してみる。

$$
p(w_t|w_{<t}, c)
$$

これは、なんらかの条件$c$に基づいた文章を生成するモデルと見られる。

例えば条件を画像とする場合、画像に基づいた文章を生成するモデルとなり、画像のキャプション生成などに使える。また条件を音声とする場合、音声に基づいた文章を生成するモデルとなり、音声認識などに使える。

では、文章を条件とする場合を考えてみる。この場合、文章から文章を生成するモデルとなる。これはどんなことに使えるだろう。

例えば文章要約が挙げられる。条件としてある文章を与え、そこから要点のみをまとめた新たな文章を生成する。また、機械翻訳も考えられそう。入力された文章から、同じ意味を持った別の言語の文章を生成する。

### RNNへの条件付け

RNNがこれらの条件を考慮するためにはどうすればよいだろうか。

といっても、条件から適当に特徴量を抽出し、それをRNNのどこかに繋げるだけでよい。適当なところで、足したり、結合して線形変換したり、やりようはいくらでもある。また、隠れ状態の初期値として条件を与える方法も考えられる。これまで0ベクトルとしていたところに、条件から抽出した特徴量を与える。

ここで、特徴抽出モデルをNNとすると、当然そのNNまで勾配が届くので、RNNと同時に学習することができる。実際に、CNNとRNNを繋げた画像のキャプション生成モデルが提案されている[1]。CNNで抽出した画像特徴量をRNNに隠れ状態の初期値として与えている。

[1] [Show and Tell: A Neural Image Caption Generator](https://arxiv.org/abs/1411.4555)


---

## Seq2Seq

*Sequence to Sequence*。*Encoder-Decoder Model*とも。

これまでRNNを言語モデルとして使ってきたが、RNNにはもう少しできることがある。それは特徴抽出である。ある時系列データを入力したときに得られる最後の隠れ状態には全ての時刻の情報が含まれており、これは特徴量と見ることが出来る。

ここで、時系列データを言語モデルへの条件として扱うことを考える。前節より、言語モデルをRNNとすると、隠れ状態の初期値として条件を与えることができる。そして時系列データの特徴抽出にはRNNが使えるため、最終的に2つのRNNを繋げたモデルができる。このモデルは時系列データ（Sequence）から時系列データへの変換を行うモデルと見られ、**Sequence to Sequence**または**Seq2Seq**と呼ばれる。

Seq2Seqは時系列データからの特徴抽出を行うRNNと時系列データを生成するRNNに分かれている。前者を**Encoder**、後者を**Decoder**と呼ぶ。ここから、Seq2Seqは**Encoder-Decoderモデル**とも呼ばれる。

Seq2Seqを用いることで時系列データから時系列データの生成が可能になる。言語モデルと関連した例を挙げると、機械翻訳や文章要約などがある。

本章では機械翻訳モデルを実装し、Seq2Seqを学ぶ。


---

## 教師データの作成

翻訳モデルの学習に必要な教師データを作成する。

### 対訳コーパス

翻訳モデルを作るには、同じ意味を持つ文章が複数の言語でまとまっているデータが必要。このようなデータは対訳コーパスと呼んだりする。

本章では以下のデータセットから日本語と英語の対訳コーパスを使用する。

- [iwslt2017  |  TensorFlow Datasets](https://www.tensorflow.org/datasets/community_catalog/huggingface/iwslt2017?hl=ja#iwslt2017-en-ja)

In [3]:
ds = tfds.load(
    "huggingface:iwslt2017/iwslt2017-en-ja",
    data_dir="data",
    split="train"
)
ds = list(ds.as_numpy_iterator())

In [4]:
data_ja = []
data_en = []
for sample in ds:
    ja = sample["translation"]["ja"].decode()
    en = sample["translation"]["en"].decode()
    data_ja.append(ja)
    data_en.append(en)

print("num of data:", len(data_ja), "\n")
for _ in range(5):
    i = random.randint(0, len(data_ja))
    print(data_ja[i])
    print(data_en[i])
    print()

num of data: 223108 

内包エネルギーを考慮しなければ 改良した家と比べると 元を取るのに 50年以上かかります これに どんな意味があるのでしょう？
Now, if I hadn't paid attention to embodied energy, it would have taken us over 50 years to break even compared to the upgraded house. So what does this mean?

予約金も返還を求められ 誰も近寄ろうとしません
Everybody wanted their deposit back. Everybody is fleeing.

最近私達が研究しているのは ハチが植物から収集する樹脂です 最近私達が研究しているのは ハチが植物から収集する樹脂です
And more recently, we've been studying resins that bees collect from plants.

ノース・アイダホ滞在中に 赤いキャンピングカーに置いていた ノートパッドを使って数えてみると
In North Idaho, in my red pickup truck, I kept a notepad.

鑑別したいものが非常にたくさんあります
There are an awful lot of things that you'd like to distinguish among.



書き出し

In [5]:
textfile_ja = "data/iwslt2017_ja.txt"
with open(textfile_ja, "w") as f:
    f.write("\n".join(data_ja))

textfile_en = "data/iwslt2017_en.txt"
with open(textfile_en, "w") as f:
    f.write("\n".join(data_en))

読み込み

In [3]:
textfile_ja = f"data/iwslt2017_ja.txt"
textfile_en = f"data/iwslt2017_en.txt"

with open(textfile_ja) as f:
    data_ja = f.read().splitlines()
with open(textfile_en) as f:
    data_en = f.read().splitlines()

#### 前処理

日本語、英語別々にトークナイザを作成する。

In [4]:
tokenizer_prefix_ja = f"models/tokenizer_iwslt2017_ja"
tokenizer_prefix_en = f"models/tokenizer_iwslt2017_en"
pad_id = 3
vocab_size = 8000

In [ ]:
spm.SentencePieceTrainer.Train(
    input=textfile_ja,
    model_prefix=tokenizer_prefix_ja,
    vocab_size=vocab_size,
    pad_id=pad_id
)

spm.SentencePieceTrainer.Train(
    input=textfile_en,
    model_prefix=tokenizer_prefix_en,
    vocab_size=vocab_size,
    pad_id=pad_id
)

In [5]:
sp_ja = spm.SentencePieceProcessor(f"{tokenizer_prefix_ja}.model")
sp_en = spm.SentencePieceProcessor(f"{tokenizer_prefix_en}.model")

unk_id = sp_ja.unk_id()
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()
pad_id = sp_ja.pad_id()

n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
print("num of vocabrary (ja):", n_vocab_ja)
print("num of vocabrary (en):", n_vocab_en)

num of vocabrary (ja): 8000
num of vocabrary (en): 8000


トークン化

In [6]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

BOS, EOSの追加

In [7]:
for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_en.insert(0, bos_id)
    ids_ja.append(eos_id)
    ids_en.append(eos_id)

### 教師データの作成

入力文と正解のペアを作成する。

通常のRNNLMでは、トークン列とそれを1つずらしたトークン列がペアとなる。Seq2SeqではこのペアがDecoderへの入力と正解となり、これに加えてEncoderへの入力（条件）を用意する。

例）

入力（Encoder） | 入力（Decoder） | 正解
--- | --- | ---
夏 休み が 終わり ました 。 \<EOS> | \<BOS> Summer vacation is over . | Summer vacation is over . \<EOS>
ツイッター は 亡くなり ました 。 \<EOS> | \<BOS> Twitter is dead . | Twitter is dead . \<EOS>
今日 から X で 暮らし ましょう 。 \<EOS> | \<BOS> Let 's live in X from today . | Let 's live in X from today . \<EOS>

`DataLoader`の作成。

In [8]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja # Encoderへの入力
        x_dec = en[:-1] # Decoderへの入力
        y_dec = en[1:] # Decoderの出力
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

def collate_fn(batch): # padding
    x_enc, x_dec, y_dec = zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

dataset = TextDataset(data_ids_ja, data_ids_en)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])
print("num of train data:", len(train_dataset))
print("num of test data:", len(test_dataset))

batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    drop_last=True,
    collate_fn=collate_fn
)

x_enc, x_dec, y_dec = next(iter(train_loader))
x_enc.shape, x_dec.shape, y_dec.shape

num of train data: 178487
num of test data: 44621


(torch.Size([32, 45]), torch.Size([32, 51]), torch.Size([32, 51]))


---

## 双方向RNN

*Bidirectional RNN*

順方向と逆方向の両方で演算を行うRNN。

Seq2SeqのEncoderのような、特徴抽出器としてのRNNに活用できる。

これまでのRNNでは、系列長$T$の入力$x_1, x_2, \cdots, x_T$に対して時刻$t=1,2,\cdots,T$の順に演算を行う。これを順方向の演算と呼ぶことにする。これに対し、時刻$t=T,T-1,\cdots,1$の順に行う演算を考え、これを逆方向の演算と呼ぶことにする。

双方向RNNでは、順方向の演算に加え逆方向の演算も行い、各時刻$t$で2つの隠れ状態$\boldsymbol h_t^{(f)}, \boldsymbol h_t^{(b)}\in\mathbb R^d$を出力する。これらの隠れ状態を結合した

$$
\boldsymbol h_t = \begin{pmatrix}
\boldsymbol h_t^{(f)} \\
\boldsymbol h_t^{(b)}
\end{pmatrix} \in\mathbb R^{2d}
$$

を最終的な出力とすることが多いかな。

逆方向演算には別のパラメータを用いるので、パラメータ数は二倍に増える。またLSTMやGRUでも同じことができる。

この双方向RNNは、特徴抽出としてのRNNで大きな力を発揮する。$t$より前の入力$x_{<t}$だけでなく、$t$より後の入力$x_{>t}$も考慮して隠れ状態$h_t$を出力するため、$h_t$は入力シーケンス全体が考慮された隠れ状態となる。当然この方が表現力が上がる。

固定長の隠れ状態が欲しい場合は最後の隠れ状態を取得すれば良い。最後の隠れ状態も順方向と逆方向の二種類$h_T^{(f)}, h_1^{(b)}$が存在するので、それらを結合して使うと良いね。

なお、特徴抽出のために使うことは出来るが、文章生成のためには使えない。文章生成中は$t$より後の情報がないから。

実装してみようか。

In [9]:
class BidirectionalRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn_cell_forward = nn.RNNCell(input_size, hidden_size)
        self.rnn_cell_backward = nn.RNNCell(input_size, hidden_size)

    def forward(self, x, h_forward, h_backward):
        # x: (seq_len, batch_size, input_size)
        # h_forward: (batch_size, hidden_size)
        # h_backward: (batch_size, hidden_size)

        hs_forward = []
        hs_backward = []

        # 順方向
        for xt in x:
            h_forward = self.rnn_cell_forward(xt, h_forward)
            hs_forward.append(h_forward)

        # 逆方向
        for xt in reversed(x):
            h_backward = self.rnn_cell_backward(xt, h_backward)
            hs_backward.insert(0, h_backward)

        hs_forward = torch.stack(hs_forward)
        hs_backward = torch.stack(hs_backward)
        hs = torch.cat([hs_forward, hs_backward], dim=-1)
            # (seq_len, batch_size, hidden_size * 2)

        return hs, (h_forward, h_backward)

`nn.RNNCell`を順方向用と逆方向用に二つ用意し、それらを使って各時刻の隠れ状態を求める。時刻ごとに得られる二種類の隠れ状態を`torch.cat`で結合して出力する。

In [10]:
seq_len, batch_size, input_size, hidden_size = 10, 32, 128, 128
x = torch.randn(seq_len, batch_size, input_size)
h_forward = torch.zeros(batch_size, hidden_size)
h_backward = torch.zeros(batch_size, hidden_size)

birnn = BidirectionalRNN(input_size, hidden_size)
hs, (h_forward, h_backward) = birnn(x, h_forward, h_backward)
hs.shape, h_forward.shape, h_backward.shape

(torch.Size([10, 32, 256]), torch.Size([32, 128]), torch.Size([32, 128]))

### PyTorchでの実装

`bidirectional=True`でOK。

In [11]:
birnn = nn.RNN(input_size, hidden_size, bidirectional=True)
hs, h = birnn(x)
hs.shape, h.shape

(torch.Size([10, 32, 256]), torch.Size([2, 32, 128]))

最後の隠れ状態はstackされて一つの`Tensor`として出力される。

パラメータが二倍になることも確認できる。

In [12]:
n_params_birnn = sum(p.numel() for p in birnn.parameters())

rnn = nn.RNN(input_size, hidden_size)
n_params_rnn = sum(p.numel() for p in rnn.parameters())

print("num of parameters (RNN):", n_params_rnn)
print("num of parameters (BiRNN):", n_params_birnn)

num of parameters (RNN): 33024
num of parameters (BiRNN): 66048


基本的には`bidirectional=True`とするだけで良いが、一部のケースではもう少しいじる必要がある。

双方向RNNの逆方向の演算は入力シーケンスの最後から始まる。ここで、入力シーケンスがpaddingされている場合、padding部分を除いた位置から演算が開始されて欲しい。これを実現するために、`pack_padded_sequence`を使う。

長さの違うサンプルとそれをpaddingしたデータがあったとする。

In [13]:
x = [
    torch.tensor([1, 2, 3]),
    torch.tensor([1, 2]),
    torch.tensor([1, 2, 3, 4, 5]),
]
padded_x = pad_sequence(x, batch_first=True, padding_value=0)
padded_x

tensor([[1, 2, 3, 0, 0],
        [1, 2, 0, 0, 0],
        [1, 2, 3, 4, 5]])

これを埋め込む。

In [14]:
embed = nn.Embedding(10, 2)
z = embed(padded_x)
z.shape

torch.Size([3, 5, 2])

さて、これをRNNに入力するわけだが、そのまま与えるとpadding部分も計算に含まれてしまう。これを避けるために、`PackedSequence`というオブジェクトを使う。`pack_padded_sequence`関数にpaddingされたデータとpaddingされていない部分の長さのリストを与えることで得られる。

In [15]:
from torch.nn.utils.rnn import pack_padded_sequence

In [16]:
lengths = list(map(len, x)) # 各系列の長さ
print(lengths)

packed_x = pack_padded_sequence(
    z, lengths, batch_first=True, enforce_sorted=False
)
packed_x

[3, 2, 5]


PackedSequence(data=tensor([[-0.1484, -0.1109],
        [-0.1484, -0.1109],
        [-0.1484, -0.1109],
        [-0.3817,  0.6437],
        [-0.3817,  0.6437],
        [-0.3817,  0.6437],
        [-0.1909,  0.8276],
        [-0.1909,  0.8276],
        [-2.4133, -0.7248],
        [-0.3764,  0.3806]], grad_fn=<PackPaddedSequenceBackward0>), batch_sizes=tensor([3, 3, 2, 1, 1]), sorted_indices=tensor([2, 0, 1]), unsorted_indices=tensor([1, 2, 0]))

`PackedSequence`というオブジェクトが取得できた。これをRNNに入力すると、paddingされた部分が無視される。

In [17]:
birnn = nn.RNN(2, 2, bidirectional=True, batch_first=True)
packed_hs, h = birnn(packed_x)

出力も`PackedSequence`。

In [18]:
print(type(packed_hs))
packed_hs

<class 'torch.nn.utils.rnn.PackedSequence'>


PackedSequence(data=tensor([[-0.2367, -0.3998, -0.5968, -0.3985],
        [-0.2367, -0.3998, -0.4693, -0.3490],
        [-0.2367, -0.3998, -0.3869, -0.3588],
        [ 0.1080, -0.6643, -0.7101, -0.0655],
        [ 0.1080, -0.6643, -0.4663,  0.0085],
        [ 0.1080, -0.6643, -0.3728,  0.1479],
        [ 0.0237, -0.6648, -0.7310, -0.0814],
        [ 0.0237, -0.6648, -0.2600,  0.1858],
        [-0.3297, -0.2939, -0.9617, -0.1502],
        [ 0.0584, -0.6101, -0.3632,  0.0389]], grad_fn=<CatBackward0>), batch_sizes=tensor([3, 3, 2, 1, 1]), sorted_indices=tensor([2, 0, 1]), unsorted_indices=tensor([1, 2, 0]))

`pad_packed_sequence`で`Tensor`に戻す。

In [19]:
from torch.nn.utils.rnn import pad_packed_sequence

In [20]:
hs, lengths = pad_packed_sequence(packed_hs, batch_first=True, padding_value=0)
hs

tensor([[[-0.2367, -0.3998, -0.4693, -0.3490],
         [ 0.1080, -0.6643, -0.4663,  0.0085],
         [ 0.0237, -0.6648, -0.2600,  0.1858],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[-0.2367, -0.3998, -0.3869, -0.3588],
         [ 0.1080, -0.6643, -0.3728,  0.1479],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[-0.2367, -0.3998, -0.5968, -0.3985],
         [ 0.1080, -0.6643, -0.7101, -0.0655],
         [ 0.0237, -0.6648, -0.7310, -0.0814],
         [-0.3297, -0.2939, -0.9617, -0.1502],
         [ 0.0584, -0.6101, -0.3632,  0.0389]]],
       grad_fn=<IndexSelectBackward0>)

`lengths`には長さのリストが入っている。

In [21]:
lengths

tensor([3, 2, 5])

最後の隠れ状態は普通に`Tensor`で返ってくる。

In [22]:
type(h), h.shape

(torch.Tensor, torch.Size([2, 3, 2]))

余談。

`PackedSequence`を使うことでpaddingされた部分が無視される。先では双方向RNNのためにこの機能を使ったが、単方向のRNNのための機能でもある。最後の隠れ状態にpaddingされた部分が含まれないようにするために使える。「padding部分を除いた最後の隠れ状態」が欲しい場合に使う。

一応全ての隠れ状態は得られるので、padを除いた最後の位置が分かればそこを指定して取り出すこともできる。ただRNNの章で述べた通り、厳密には最後の層からの出力しか得られないため、`num_layers`を2以上として複数のRNN層を重ねる場合、全ての層のpadを除いた最後の隠れ状態を得るためには`PackedSequence`を使うしかない。


---

## ビームサーチ

*Beam Search*

言語モデルを用いた様々な文章生成手法を説明する。最終的にビームサーチと呼ばれる手法を学び、後の節・章で活用する。

これまで、言語モデルが出力した確率分布からのサンプリングによって次の単語を予測し、文章を生成した。しかし、この手法では、仮に上手く分布を予測できても、乱数によって変な文章が生成される可能性がある。softmaxの性質上、単語の確率が0になることはないので、どんな単語でも選ばれる可能性があるのだ。当然低い確率を設定できればそれが選ばれる可能性は低くなるが、適切でない単語全てに<u>0と見ても問題ないと言える程の低い確率</u>を割り当てられるように学習することは現実的でない。

本節では、そういった問題を考慮した文章生成方法を説明する。シンプルな手法が多いので、どんな手法があるかを予想してから読み進めてもいいかもしれない。

余談

この問題は翻訳に限らず、言語モデルを用いた文章生成に対して一般的に言えるものである。それをなぜ初めに説明せずにこのタイミングで説明しているか。

一つは、初めに色々な内容を詰め込む必要もないかなと思ったから。本質的な部分じゃないし、後でいいかなって。

もう一つは、翻訳にはある程度の正解が定められており、そこに近づくためにはより正確な文章生成が求められるから。ここでやるのがちょうどいいかなって。

### Top-k

確率の高い単語を上から$k$個抽出し、その中からランダムで一つ選択する方法。確率の低い単語を選択肢から除くことが出来る。ランダムで選ぶ際の確率分布は各単語の確率によって決める。

例えば

単語 | 確率
--- | ---
私 | 0.2
おいしい | 0.4
今日 | 0.1
太陽 | 0.1
です | 0.2

という確率分布に対し$k=3$でTop-kによる選択を行う場合、確率の高い3つの単語「私」、「おいしい」、「です」からランダムに選ぶことになる。この時の確率分布は0.2, 0.4, 0.2を正規化した0.25, 0.5, 0.25となる。


単語 | 確率
--- | ---
私 | 0.25
おいしい | 0.5
です | 0.25

### Top-p

確率の合計が$p$を超えるまでの単語を抽出する。それ以降はTop-kと同じ。

例えば

単語 | 確率
--- | ---
私 | 0.2
おいしい | 0.3
今日 | 0.1
太陽 | 0.1
です | 0.3

という確率分布に対し$p=0.5$でTop-pによる選択を行う場合、「おいしい」、「です」が抽出される。

### 貪欲法

greedy法とも。各時刻で確率が最も高い単語を選び続ける手法。$k=1$のTop-kや$p=0$のTop-pとも見られる。

貪欲法は各時刻で最適解を取り続けるが、全体で見たときに最適なものが得られているかは分からない。例えば、貪欲法によってシーケンス$w_1,w_2$を取得し、その時の確率$p_1,p_2$が$0.9,0.3$であったとする。しかし、もしかしたら$p_1,p_2=0.8,0.8$となるような$w_1,w_2$が存在したかもしれない。$0.9\times0.3=0.27$に対し$0.8\times0.8=0.64$なので後者の方がシーケンス全体で見たときの確率は高い。貪欲法ではそれを取得できない。

### 全探索

考えられる全てのシーケンスに対して確率を求め、最も確率の高いシーケンスを選ぶ。貪欲法の課題を解決できる。

ただ、単語数に制限を設けない場合考えられるシーケンスの数が無限となり、仮に制限を設けたとしてもその数は指数的に増加するので、計算量の問題でこの手法は現実的でない。この話一番初めにもしたね。

### ビームサーチ

Top-kと貪欲法と全探索をいい感じに混ぜ合わせたもの。

まず初めの単語を確率の高い順に$k$個選ぶ。次に、選んだ$k$個の単語それぞれについて、次に続く単語を上からk個選ぶ。この時点で$k^2$個の単語列が候補として存在する。ここで、その単語列の中から確率の高いものを$k$個選ぶ。以後繰り返し。


---

## 実践

実際にSeq2Seqを学習させて翻訳を行ってみる。

### モデル構築

Encoder、Decoderを作成し、Seq2Seqモデルを作成する。

#### Encoder

入力文を入れて固定長のベクトルを出力するRNN。

これまでのRNN同様、適当にLSTMと線形層で作る。前節で説明した双方向LSTMを取り入れ、さらに精度向上のため、以下の工夫を加える。

- LSTMを3層に増やす
- 残差結合を取り入れる

多層化や残差結合はSeq2Seq固有の工夫ではなく、一般的なRNNに応用できる。例えば前章までのRNNLMにも適用でき、精度向上が期待される。

双方向にするのは初めの二層のみとする。三層目は単方向とし、この最後の隠れ状態をDecoderに渡す。また隠れ状態の形状の関係で残差結合は二層目だけ。

まずpackとLSTMをまとめた層を作っておく。

In [9]:
class PackedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, bidirectional=False):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            batch_first=True,
            bidirectional=bidirectional,
        )

    def forward(self, x, lengths):
        x_pack = pack_padded_sequence(
            x, lengths, batch_first=True, enforce_sorted=False
        )
        hs, (h, _) = self.lstm(x_pack)
        hs, _ = pad_packed_sequence(hs, batch_first=True)
        return hs, h


これを使ってEncoderを作る。

In [10]:
class Encoder(nn.Module):
    def __init__(
        self,
        n_vocab,
        embed_size,
        hidden_size,
        dropout=0.2,
    ):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm1 = PackedLSTM(embed_size, hidden_size // 2, True)
        self.lstm2 = PackedLSTM(hidden_size, hidden_size, False)
        self.lstm3 = PackedLSTM(hidden_size, hidden_size, False)
        # self.fc_skip = nn.Linear(embed_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        lengths = (x != pad_id).sum(dim=1).cpu()

        # 埋め込み層
        x = self.embedding(x) # (batch_size, seq_len, embed_size)

        # LSTM1層目
        # skip = self.fc_skip(x)
        hs, _ = self.lstm1(x, lengths) # (batch_size, seq_len, hidden_size * 2)
        # hs = hs + skip
        hs = self.dropout(hs)

        # LSTM2層目
        skip = hs
        hs, _ = self.lstm2(hs, lengths) # (batch_size, seq_len, hidden_size * 2)
        hs = hs + skip
        hs = self.dropout(hs)

        # LSTM3層目
        _, h = self.lstm3(hs, lengths) # (batch_size, seq_len, hidden_size)

        return h

3つのLSTMを用意した。初めの二層は双方向。双方向RNNは二つの隠れ状態を結合して出力するので、出力する隠れ状態の次元を半分にして、結合したときに元の次元と揃うようにした。次元数が奇数の場合は想定していない。大体2の累乗だしいいでしょ。

演算にはドロップアウトや残差結合を取り入れた。入力`x`は`Tensor`ではなく`Tensor`のリスト。

余談。

残差結合は2層目だけにしている。2層目以外は層の前後の隠れ状態の形状が違うから。ただ適当な全結合層などで調整すれば揃えられるので、2層目以外に取り入れられないという訳ではない。やらないのは単にモデルが複雑になって分かり辛いからというだけ。

最後のLSTMを双方向にしていないのも同じ理由。双方向にすると隠れ状態が2種類出力されるので、Decoderに渡すことを考えると全結合層を挟んで調整する必要がある。もしくはDecoderで扱う隠れ状態の次元を倍にするか。いずれにしても複雑になるので避けている。

#### Decoder

Encoderから出力された隠れ状態を受け取り、出力文を生成するRNN。Encoder同様3層のLSTMに残差結合を取り入れ、最後に線形層を加える。

In [11]:
class Decoder(nn.Module):
    def __init__(
        self,
        n_vocab,
        embed_size,
        hidden_size,
        dropout=0.2,
    ):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm1 = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)
        # self.fc_skip = nn.Linear(embed_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, hc):
        hc1, hc2, hc3 = hc

        # 埋め込み層
        x = self.embedding(x) # (batch_size, seq_len, embed_size)

        # LSTM1層目
        # skip = self.fc_skip(x)
        hs, hc1 = self.lstm1(x, hc1) # (batch_size, seq_len, hidden_size)
        # hs = hs + skip
        hs = self.dropout(hs)

        # LSTM2層目
        skip = hs
        hs, hc2 = self.lstm2(hs, hc2)
        hs = hs + skip
        hs = self.dropout(hs)

        # LSTM3層目
        skip = hs
        hs, hc3 = self.lstm3(hs)
        hs = hs + skip
        hs = self.dropout(hs)

        # 線形層
        y = self.fc(hs) # (batch_size, seq_len, n_vocab)

        return y, (hc1, hc2, hc3)

`hc`には各LSTM層に対応する隠れ状態がタプルで与えられることを想定している。

余談。

1層目には残差結合を取り入れていない。その理由はEncoder同様層の前後で隠れ状態の形状が違うから。ただ、ここでは`embed_size`と`hidden_size`に同じ値を入れるので、できちゃうんだけどね。まあ一応変数を分けているので、矛盾が生じないように。

#### Seq2Seq

EncoderとDecoderを合わせて、入力から出力までの一連の処理を行うモデルを作る。

In [12]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        h = self.encoder(x_enc)
        hc = self.get_hc(h)
        y, _ = self.decoder(x_dec, hc)
        return y

    def get_hc(self, h):
        h = torch.stack([h] + [torch.zeros_like(h) for _ in range(2)], dim=0)
        c = torch.zeros_like(h)
        hc = zip(h, c)
        return hc

Encoderから受け取った`h`から`hc`（`(hc1, hc2, hc3)`）を作成し、Decoderに渡す。なお`hcn`は`(hn, cn)`で`h2`、`h3`、`cn`は0ベクトル。`h1`はEncoderから受け取った隠れ状態。

In [13]:
hidden_size, embed_size = 512, 512
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
model_path = "models/lm_seq2seq.pth"
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 24,379,200


余談。

EncoderからDecoderに渡すベクトルとしてpadを除いた最後の隠れ状態を採用したが、実はそれ以外にもいくつか選択肢がある。

Decoderに渡したいベクトルとして満たしてほしい条件は以下である。

- 全ての入力を参照して出力されている
- 固定長

これらを全て満たしていれば何でもよい。例えば、「padを除いた全ての隠れ状態の平均」とか。この発想は実際に使われていて、後のAttentionは全ての隠れ状態の加重平均を取ったりする。

また、padを含めて学習させるという発想もある。padを含めて学習させたら<u>padは要らない情報だ</u>と学習するからいいんじゃね、みたいな考えが出来そう。ただ実はこれはうまくいかない。padの数が多くなるにつれて隠れ状態がある一定の値に収束してしまう（経験談）。RNNに同じトークンを何度も入力することで隠れ状態が収束してしまうみたい。そうなってしまうと、入力文に依る隠れ状態の違いが少なくなり、入力文と出力の依存関係を学習できない。

あとは、全てのLSTMの最後の隠れ状態を渡すこともできる。特に今回実装したEncoderとDecoderはLSTMの数が同じなので、Encoderのn層目のLSTMをDecoderのn層目のLSTMを繋げて隠れ状態を渡すことが出来る。次元数が違う場合は適当な線形層とか挟んで。

まあそんな感じで、色々な選択肢があるのですが、とりあえずここでは性能と分かり易さがイイ感じになりそうな構造にしました。モデルの組み方はいくらでもあるということだけ分かってもらえれば。

### 学習

In [16]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)
def loss_fn(y, t):
    loss = cross_entropy(y.reshape(-1, n_vocab_ja), t.ravel())
    return loss

@torch.no_grad()
def eval_model(model):
    model.eval()
    losses = []
    for x_enc, x_dec, y_dec in test_loader:
        x_enc = x_enc.to(device)
        x_dec = x_dec.to(device)
        y_dec = y_dec.to(device)

        y = model(x_enc, x_dec)
        loss = loss_fn(y, y_dec)
        losses.append(loss.item())
    loss = sum(losses) / len(losses)
    ppl = math.exp(loss)
    return ppl

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x_enc, x_dec, y_dec in train_loader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.2f}", no_step=True)
        prog.memo()

In [17]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [18]:
train(model, optimizer, n_epochs=20, prog_unit=1)

 1/20: #################### 100% [00:05:20.53] ppl train: 148.72, test: 92.55 
 2/20: #################### 100% [00:05:25.11] ppl train: 85.50, test: 70.94 
 3/20: #################### 100% [00:05:23.16] ppl train: 69.21, test: 61.39 
 4/20: #################### 100% [00:05:22.90] ppl train: 60.00, test: 55.14 
 5/20: #################### 100% [00:05:22.78] ppl train: 53.43, test: 50.64 
 6/20: #################### 100% [00:05:21.69] ppl train: 48.53, test: 47.46 
 7/20: #################### 100% [00:05:24.18] ppl train: 44.82, test: 45.10 
 8/20: #################### 100% [00:05:24.14] ppl train: 41.80, test: 43.20 
 9/20: #################### 100% [00:05:14.76] ppl train: 39.30, test: 41.76 
10/20: #################### 100% [00:05:21.87] ppl train: 37.19, test: 40.63 
11/20: #################### 100% [00:05:23.01] ppl train: 35.33, test: 39.64 
12/20: #################### 100% [00:05:27.39] ppl train: 33.68, test: 38.91 
13/20: #################### 100% [00:05:25.38] ppl train: 32.23

In [19]:
torch.save(model.state_dict(), model_path)

### 翻訳

作成したモデルに日本語文を入力し、英語に翻訳して出力する。決定的な出力にする。

In [14]:
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [34]:
class Sentence:
    def __init__(self):
        self.sentence = []
        self.hc = None
        self.ll = 0.0
        self.norm_ll = 0.0
        self.finished = False
        self.next_token = None

    def update(self, token_id, log_prob):
        self.append(token_id)
        self.ll += log_prob
        self.norm_ll = self.ll / len(self.sentence)
        self.next_token = token_id

    def append(self, token_id):
        if token_id == eos_id:
            self.finished = True
        self.sentence.append(token_id)

    def __len__(self):
        return len(self.sentence)

    def __iter__(self):
        return iter(self.sentence)

In [35]:
bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str, # 入力文（日本語）
    lim_len: int = 100, # 出力のトークン数の上限
    k: int = 3, # ビーム幅
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor(in_ids + [eos_id], device=device).unsqueeze(0)

    h_enc = model.encoder(in_ids)
    hc = model.get_hc(h_enc)

    first_sentence = Sentence()
    first_sentence.hc = hc
    first_sentence.next_token = bos_id
    sentences = [first_sentence]
    max_len = 1

    while not all(s.finished for s in sentences) and max_len < lim_len:
        new_sentences = []
        for sentence in sentences:
            if sentence.finished:
                new_sentences.append(sentence)
                continue
            x = torch.tensor([[sentence.next_token]], device=device)
            y, hc = model.decoder(x, sentence.hc)
            y = F.log_softmax(y, dim=-1).squeeze(0, 1)
            sentence.hc = hc
            for token_id in y.topk(k).indices.tolist():
                new_sentence = copy.deepcopy(sentence)
                new_sentence.update(token_id, y[token_id].item())
                new_sentences.append(new_sentence)
        sentences = sorted(new_sentences, key=lambda s: s.norm_ll, reverse=True)
        sentences = sentences[:k]
        max_len = max(map(len, sentences))

    best_sentence = max(sentences, key=lambda s: s.norm_ll).sentence
    return sp_en.decode(best_sentence)

まずは訓練データから。

In [37]:
n = 5
for _ in range(n):
    i = random.randint(0, len(train_dataset))
    x, _, t = train_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print("input:", x)
    print("output:", translate(model, x))
    print("answer:", t)
    print()

input: 妊娠したくなかったのに 出産の際に命を落とす― 女性の数は年間10万人です
output: And if you're pregnant women's workforce, if you're pregnant women's employment, you'll notice that 80 percent of women will be diagnosed with HIV ⁇ AIDS.
answer: There are 100,000 women  ⁇ per year ⁇  who say they don't want to be pregnant and they die in childbirth -- 100,000 women a year.

input: 24時間コンテストです
output: It's 3D printers.
answer: It turned into a 24-hour contest.

input: 海というのは おそらく生命誕生の確率が最も高い場所です 地球と同じことです だからユーロパの氷の下を ぜひ探査したいのです 海で何が泳ぎまわっているのか見てみたいですし 魚や 海草や 海の怪獣がいるのか見てみたいのです 何がいても興奮します イカやタコなどの頭足動物でもいいです
output: So what's going on here is that we've gone down to Africa's largest metropolitan area where we're going to be able to eat ice cream, but if you're lucky enough to get rid of the rooftops, you'll notice that there's a lot of toxins coming out of nowhere, and it turns out there's a lot of toxins involving a lot of flowers. It's a lot easier. It doesn't look like
answer: Ocean -- probably the most likely place

あまりよくないね。

訓練データに含まれていないものも試してみる。

In [38]:
# test data
for _ in range(n):
    i = random.randint(0, len(test_dataset))
    x, _, t = test_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print("input:", x)
    print("output:", translate(model, x))
    print("answer:", t)
    print()

input: 僕らは 母親に腹を立てていた 窓の外には


output: We've been blessed for granted, "I'm sorry."
answer: We were really mad at my mom. We looked out the window.

input: 消費者は 共同組合を形成しました 画期的な例として カリフォルニアのキャロット・モブ運動があります
output: And they're called "The Little Millennium Development Goals."
answer: And then there is this other really interesting movement that's happened in California, which is about carrot mobs.

input: そんな行動の根源には 何があるのでしょう?
output: So how do we deal with uncertainty? What's happening?
answer: What lay at the root of their behavior?

input: 始めるにあたって 3Dプリンターを用意し ビーカーや試験管などの 実験容器を印刷しました 同時に別のプリンターで 分子を印刷し 「反応容器」の中で 組み合わせました
output: And so what we did was we took three weeks ago, and we looked at the Nanopatch, and then we put together a bunch of algorithms called Planets, and then we found out how to manipulate neurological disorder. And then we found ourselves. We've got to get rid of the robot.
answer: Well to start to do this, we took a 3D printer and we started to print our beakers and our test tubes on one side 

In [40]:
# original
sentences = [
    "ありがとう",
    "私はかわいい猫を飼っています。",
    "上手く文章が書けるようになりました"
]

for sentence in sentences:
    print("input:", sentence)
    print("output:", translate(model, sentence))
    print()

input: ありがとう
output: Thank you.

input: 私はかわいい猫を飼っています。
output: And I'm sorry. There's lots of flowers.

input: 上手く文章が書けるようになりました
output: And then I started writing letters.



びみょう。